### Pranay Sharma

In [2]:
import tensorflow as tf

In [17]:
def downsample(x,filter):
    res_x = x
    x = tf.keras.layers.Conv2D(
        filters=filter,
        kernel_size=3,
        strides=1,
        padding='same',
        use_bias=True
        )(x)
    
    x = tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=(2, 2),
        padding='same'
        )(x)
    
    res_X = tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=(2, 2),
        padding='same'
        )(res_x)
    
    x = tf.keras.layers.Concatenate(axis=-1)([x, res_X]) # approach 1
    x = tf.keras.layers.Conv2D(
        filters=filter,
        kernel_size=1,
        padding='same',
        use_bias=True
        )(x)
    
    # res_x = tf.keras.layers.Conv2D(
    #     filters=filter,
    #     kernel_size=1,
    #     padding='same',
    #     use_bias=True
    #     )(res_x)

    # x = tf.keras.layers.Add()([x, res_X]) # approach 2
    

    x = tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.9)(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x



In [14]:
def resNet():
    inputs = tf.keras.Input(shape=(256, 256, 3))
    x1 = tf.keras.layers.Conv2D(
        filters=16,
        kernel_size=3,
        padding='same',
        use_bias=True
    )(inputs)
    x1 = tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.9)(x1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    
    x2 = downsample(x1, 64) #output shape (128, 128, 64)
    x3 = downsample(x2, 128) # output shape (64, 64, 128)
    x4 = downsample(x3, 256) # output shape (32, 32, 256)
    x5 = downsample(x4, 512) # output shape (16, 16, 512)
    x6 = downsample(x5, 1024) # output shape (8, 8, 1024)
    x7 = downsample(x6, 2048) # output shape (4, 4, 2048)
    x8 = tf.keras.layers.GlobalAveragePooling2D()(x7) # output shape (2048,)

    x9 = tf.keras.layers.Dense(1000, activation='relu')(x8) # assuming 1000 classes for classification
    x10 = tf.keras.layers.Dense(1000, activation='sigmoid')(x9) # final output layer
    reshape_layer = tf.keras.layers.Reshape((250,4))(x10) # reshape to (250, 4)
    model = tf.keras.Model(inputs=inputs, outputs=reshape_layer)
    return model




In [15]:
model = resNet()

In [16]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 256, 256,  │        448 │ input_layer_4[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_19[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_11       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 256, 256,  │      9,280 │ activation_11[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_16    │ (None, 128, 128,  │          0 │ conv2d_20[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_17    │ (None, 128, 128,  │          0 │ activation_11[0]… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, 128, 128,  │          0 │ max_pooling2d_16… │
│ (Concatenate)       │ 80)               │            │ max_pooling2d_17… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 128, 128,  │      5,184 │ concatenate_8[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_21[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 128, 128,  │     73,856 │ activation_12[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_18    │ (None, 64, 64,    │          0 │ conv2d_22[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_19    │ (None, 64, 64,    │          0 │ activation_12[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_9       │ (None, 64, 64,    │          0 │ max_pooling2d_18… │
│ (Concatenate)       │ 192)              │            │ max_pooling2d_19… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 64, 64,    │     24,704 │ concatenate_9[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_23[0][0] 

 Total params: 36,610,704 (139.66 MB)

 Trainable params: 36,602,608 (139.63 MB)

 Non-trainable params: 8,096 (31.62 KB)